In [2]:
import pandas as pd
import sys
sys.path.append('..')
from functions.select_cols_functions import *

import itertools
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np
import random

import warnings
from sklearn.exceptions import UndefinedMetricWarning

In [34]:
# Ignorar UndefinedMetricWarning
warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

In [35]:
df = pd.read_csv('../../../data/train_2_classes.csv')

In [36]:
df = apply_all_rules(df)

remover_colunas = [
    'Name',
    'RescuerID'
]

df = df.drop(columns=remover_colunas)

In [37]:
# df = df[df['Quantity'] == 1]

In [38]:
df.columns

Index(['Type', 'Age', 'Breed1', 'Breed2', 'Gender', 'Color1', 'Color2',
       'Color3', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed',
       'Sterilized', 'Health', 'Quantity', 'Fee', 'State', 'VideoAmt',
       'PhotoAmt', 'AdoptionSpeed'],
      dtype='object')

In [39]:
# Colunas de interesse (sem incluir a target)
columns = ['Type', 'Age', 'Breed1', 'Breed2', 'Gender', 'Color1', 'Color2',
           'Color3', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed',
           'Sterilized', 'Health', 'Quantity', 'Fee', 'State', 'VideoAmt', 'PhotoAmt']

# Coluna target
target = 'AdoptionSpeed'

In [40]:
# Limite o número máximo de colunas por combinação (exemplo: 5 colunas)
max_features = 5

# Gerar as combinações apenas até o limite definido
combinacoes_limitadas = []
for i in range(1, max_features + 1):
    combinacoes_limitadas.extend(itertools.combinations(columns, i))

print(f"Total de combinações limitadas a {max_features} colunas: {len(combinacoes_limitadas)}")


Total de combinações limitadas a 5 colunas: 16663


In [41]:
# Número de combinações que você deseja testar aleatoriamente
num_combinations = 1000

# Gerar uma amostra aleatória de combinações
random_combinations = random.sample(list(itertools.chain.from_iterable(itertools.combinations(columns, i) for i in range(1, len(columns) + 1))), num_combinations)

print(f"Total de combinações aleatórias selecionadas: {len(random_combinations)}")


Total de combinações aleatórias selecionadas: 1000


In [42]:
# Usar todas as colunas para treinar um modelo de Random Forest e avaliar a importância das features
X = df[columns]
y = df[target]

model = RandomForestClassifier()
model.fit(X, y)

# Importância das features
importances = model.feature_importances_

# Ordenar as colunas pela importância
colunas_importantes = [col for _, col in sorted(zip(importances, columns), reverse=True)]

# Exibir as colunas mais importantes
print("Colunas mais importantes:", colunas_importantes[:10])  # Limitar as 10 mais importantes

# Gerar combinações usando apenas as colunas mais importantes
combinacoes_limitadas_importantes = []
for i in range(1, 6):  # Limitar até 5 colunas
    combinacoes_limitadas_importantes.extend(itertools.combinations(colunas_importantes[:10], i))

print(f"Total de combinações usando as colunas mais importantes: {len(combinacoes_limitadas_importantes)}")


Colunas mais importantes: ['PhotoAmt', 'Age', 'Breed1', 'Color2', 'Color1', 'State', 'Breed2', 'Quantity', 'MaturitySize', 'Fee']
Total de combinações usando as colunas mais importantes: 637


In [43]:
def train_and_evaluate(X_train, X_test, y_train, y_test):
    model = RandomForestClassifier(n_estimators=750)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    # Calcular as métricas com zero_division=0
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    
    return accuracy, f1, precision, recall

In [44]:
results = []

In [45]:
total_combinacoes = len(combinacoes_limitadas_importantes)

for idx, combo in enumerate(combinacoes_limitadas_importantes, start=1):
    # Exibir a iteração atual e o total em uma única linha (com carriage return)
    progress = f"Iteração {idx}/{total_combinacoes} - ({(idx/total_combinacoes)*100:.2f}% concluído)"
    sys.stdout.write('\r' + progress)
    sys.stdout.flush()

    # Separar as features
    X = df[list(combo)]
    
    # Dividir em treino e teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Treinar e avaliar o modelo
    accuracy, f1, precision, recall = train_and_evaluate(X_train, X_test, y_train, y_test)
    
    # Armazenar os resultados
    results.append({
        'features': combo,
        'accuracy': accuracy,
        'f1_score': f1,
        'precision': precision,
        'recall': recall
    })

Iteração 637/637 - (100.00% concluído)

In [46]:
results_df = pd.DataFrame(results)

In [47]:
results_df = results_df.sort_values(by='accuracy', ascending=False)

In [48]:
results_df.to_csv('./combination_results.csv', index=False)